In [3]:
from om_ogc_timeseries_client import OmOgcTimeseriesClient
import pandas
import plotly.express as px
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

Instantiate client

In [4]:
client = OmOgcTimeseriesClient({"token":"whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361"})

retrieve timeseries using site and variable ids

In [24]:

ts_metadata = client.getTimeseries(
    feature="FAAC49BA633EFF325BE5D2BA81BE14574A268ABA",
    observedProperty="Discharge",
    aggregationDuration="P1M")

DEBUG:root:url: https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gs-service-preproduction.geodab.eu:443
DEBUG:urllib3.connectionpool:https://gs-service-preproduction.geodab.eu:443 "GET /gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations?feature=FAAC49BA633EFF325BE5D2BA81BE14574A268ABA&observedProperty=Discharge&aggregationDuration=P1M&useCache=False&limit=1000 HTTP/1.1" 200 None


Inspect metadata

In [25]:
len(ts_metadata["member"])

3

Select observation identifier

In [26]:
observationIdentifier = ts_metadata["member"][0]["id"]

In [31]:
observationIdentifier

'18EB307E3D1C45D3A2842D710A41001AB5083041'

retrieve data

In [27]:

data = client.getData(
    "1990-01-01", 
    "2025-05-01",
    observationIdentifier = observationIdentifier)

DEBUG:root:url: https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gs-service-preproduction.geodab.eu:443
DEBUG:urllib3.connectionpool:https://gs-service-preproduction.geodab.eu:443 "GET /gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations?observationIdentifier=18EB307E3D1C45D3A2842D710A41001AB5083041&useCache=False&limit=1000 HTTP/1.1" 200 None
DEBUG:root:url: https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gs-service-preproduction.geodab.eu:443
DEBUG:urllib3.connectionpool:https://gs-service-preproduction.geodab.eu:443 "GET /gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view

convert to dataframe and plot

In [29]:
df = pandas.DataFrame(data)
df["date"] = pandas.to_datetime(df["date"])
df = df.set_index("date")
px.line(df.reset_index(), x="date", y="value")

read observation identifiers from csv, retrieve data and plot together

In [30]:
stations = pandas.read_csv("data/ina_timeseries.csv")
df_list = []
for index, row in stations.iterrows():
    data = client.getData(
        "2023-01-01",
        "2024-05-01", 
        observationIdentifier = row["observation_identifier"])
    df_ = pandas.DataFrame(data)
    df_["station_name"] = row["station_name"]
    df_list.append(df_)
df = pandas.concat(df_list)
px.line(df, x = "date", y = "value", color = "station_name")

DEBUG:root:url: https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gs-service-preproduction.geodab.eu:443
DEBUG:urllib3.connectionpool:https://gs-service-preproduction.geodab.eu:443 "GET /gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations?observationIdentifier=8272678FE72DB91CD511E653099DB3219DEE615B&useCache=False&limit=1000 HTTP/1.1" 200 None
DEBUG:root:url: https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view/whos-plata/om-api/observations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gs-service-preproduction.geodab.eu:443
DEBUG:urllib3.connectionpool:https://gs-service-preproduction.geodab.eu:443 "GET /gs-service/services/essi/token/whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361/view